# D. Pretrained embeddings

In this notebook, I used word embeddings to transform my textual data into numerical represenations. For this purpose, I used the data was cleaned in the notebook 'B. Vectorization' and that can be found in the gold_data folder. Further, I use the same base classifiers and resampling techniques that I used in the previous notebook.

First, I trained a Word2Vec model (both the continuous bag of words and the skipgram implementation) to transform my textual data. Then, I used Glove embeddings for text represenation. However, as I only have a limited dataset to train embedding models, I aslo used pretrained embedding models. Then, I could use these pretrained embeddings to transform my train and test set and train my models.

# 0. Data loading

In [1]:
# General Packages #
import os
import pandas as pd
import numpy as np
import string
import re
from scipy.stats import randint
import random
from collections import Counter

# Sklearn Packages #
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# NLTK Packages #
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob, Word
from nltk.tokenize import word_tokenize

# Import necessary libraries for handling imbalanced data
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Embedding related imports
import sys
import gensim
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.models import KeyedVectors
import gensim.downloader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.scripts.glove2word2vec import glove2word2vec

import spacy.cli

In [2]:
# Turn of warnings, just to avoid pesky messages that might cause confusion here
# Remove when testing your own code #
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Change to Working Directory with Training Data # 
#os.chdir("/Users/Artur/Desktop/thesis_HIR_versie5/coding")
os.chdir("/Users/juarel/Desktop/studies artur/thesis_HIR/coding")

# Load the preprocessed data #
df_train = pd.read_csv("./data/gold_data/train.csv", header = 0)
df_test = pd.read_csv("./data/gold_data/test.csv", header = 0)

# inspect the data
df_train.head(5)

,id,Headline,category,cleaned_headline
0,194578,Head Line: US Patent granted to BASF SE (Delaw...,None,head u patent granted se delaware may titled c...
1,564295,Societe Generale Launches a Next-Generation Ca...,None,societe generale launch nextgeneration card in...
2,504138,BARCLAYS PLC Form 8.3 - EUTELSAT COMMUNICATION...,None,plc form communication
3,91379,ASML: 4Q Earnings Snapshot,None,4q earnings snapshot
4,265750,Form 8.3 - AXA INVESTMENT MANAGERS : Booker Gr...,None,form investment manager group plc


# 1. Define functions and parameters

Before we continue, we first define some useful functions and parameters that we use throughout this notebook. The first four functions and parameters were also used and defined in the previous notebook.

1. get_classification_metrics: Create a function that return the classification metrics for each model. The precision, recall and f1 score are all determined using the average value of all classes, without adjusting weights to these classes.

2. Define a dataframe to store the results of the different models. Moreover, also define a dictionary that stores the best parameters for each model.

3. Define the number of splits, the stratified cross validator to ensure class frequencies are considered, and the scoring metric based on the average F1 score. We use an F1 score as scoring metric as accuracy is not a good evaluation metric in our case.

4. Define a function that trains the defined model, the input data, the classifier and its parameter grid. Besides, it will also take 4 parameters as input that give more information about the model that is being trained. This is usefull for the storage of the performance of the different algorithms.



New functions specific to the embedding notebook:

5. Create a function to create embeddings based on a pretrained model.

In [4]:
# 1. Function that returns classication metrics
def get_classification_metrics(y_true, y_pred):
    
    # Calculate Model Performance Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')


    return accuracy, precision, recall, f1


In [5]:
# 2. Create an empty dataframe to store the results of all the models
results_all_df = pd.DataFrame()

# Add columns for the metrics
columns = ['vectorizer', 'FS', 'classifier', 'resampling','accuracy', 'precision', 'recall', 'f1']
for col in columns:
    results_all_df[col] = 0

# create an empty dictionary to store the optimal parameters
best_params_dict = {}

In [6]:
# 3. Define different parameters
# Define the number of folds for cross-validation
n_splits = 5

# Initialize the stratified k-fold object
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) # ensures class balances are kept

# Define the scoring metric
scoring = make_scorer(f1_score, average= 'macro')

In [7]:
# 4. Define a function to train and evaluate the different models
def perform_grid_search(name, model, param_grid, X_train, X_test, y_train, y_test,
                       vectorizer, FS, classifier, resampling):
    
    # Define a seed value
    random.seed(7)
        
    # Perform the grid search using cross-validation
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring=scoring)
    grid_search.fit(X_train, y_train)

    # Get the best model and its hyperparameters
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Store the best parameters for the current category in the dictionary
    best_params_dict[name] = best_params
    print(f'best parameters: {best_params}')

    # Retrain the best model with the whole training set
    best_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_model.predict(X_test)
    
    # Calculate the probabilities (not for SVM as this is not possible)
    if classifier != 'SVM':
        y_pred_proba = best_model.predict_proba(X_test)
        
        # Find the highest probability for each observation
        highest_prob = np.amax(y_pred_proba, axis = 1)
    
        # Create a DataFrame with test observations, highest probabilities, and predicted classes
        predictions_df = pd.DataFrame({'Observation_nr': y_test.index, 'Probability': highest_prob, 'Prediction': y_pred})
        
    else:
        # Create a DataFrame with test observations and predicted classes
        predictions_df = pd.DataFrame({'Observation_nr': y_test.index, 'Prediction': y_pred})
        
    # Store the final predictions with its probability for the test set
    predictions_df.to_csv(f'./Output/predictions/{name}.csv', index = False, header = True)
    #predictions_df.to_excel(f'./Output/predictions/{name}.xlsx', index = False, header = True)

    # Calculate the classification metrics
    accuracy, precision, recall, f1 = get_classification_metrics(y_test, y_pred)
    
    # print the results
    print(f'Results for {name}:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')
    
    # add the results to the dataframe with all the results
    results_all_df.loc[name] = [vectorizer, FS, classifier, resampling, accuracy, precision, recall, f1]

In [8]:
# 5. Define a function to transform the data into embeddings based on a pretrained model
def vectorize_pretrained(headline, pre_trained_model, size):

    # Split the headline into individual words
    words = headline.split()

    # Retrieve word embedding for each word in the headline
    words_vecs = [pre_trained_model[word] for word in words if word in pre_trained_model]

    # If no word vectors are found (i.e., no matching words in the pretrained model),
    # return a zero vector of the specified dimension
    if len(words_vecs) == 0:
        return np.zeros(size)

    # Convert the list of word vectors into a numpy array and average across word vectors
    words_vecs = np.array(words_vecs)
    mean_vector = words_vecs.mean(axis=0)

    return mean_vector

In [9]:
# define the independent and dependent variables
X_train = df_train['cleaned_headline']
X_test = df_test['cleaned_headline']

y_train = df_train['category']
y_test = df_test['category']

# 3. Pretrained embeddings

In [10]:
# define vectorizer method
vectorizer = 'pretrained'

## 3.1 Google news embeddings

In [11]:
# define type of pretrained embedding model
FS = 'google news'

Uses continious skipgram: http://vectors.nlpl.eu/repository/?ref=blog.paperspace.com

https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/


In [12]:
# if downloaded offline
google_news = KeyedVectors.load_word2vec_format('/Users/juarel/Desktop/studies artur/thesis_HIR/big files/google/GoogleNews-vectors-negative300.bin', binary=True)
#google_news = KeyedVectors.load_word2vec_format('/Users/Artur/Desktop/thesis_HIR_versie5/big files/google/GoogleNews-vectors-negative300.bin', binary=True)

In [13]:
# Download the "word2vec-google-news-300" embeddings if not downloaded ofline
#google_news = gensim.downloader.load('word2vec-google-news-300')

In [14]:
# Create embeddings of the train and test set based on the pretrained google_news model
X_train_google_news = np.vstack([vectorize_pretrained(headline, google_news, 300) for headline in X_train])
X_test_google_news = np.vstack([vectorize_pretrained(headline, google_news, 300) for headline in X_test])

### 3.1.1 Without resampling

In [15]:
# Define the resampling technique
resampling = 'None'

#### A. Logistic regression

In [16]:
# define the model characteristics
model_name = 'google_log_w'
classifier = 'logR'

# Initialize the classifier
logreg = LogisticRegression(random_state = 7)

# Define the parameter grid
param_grid_log = {
    'penalty': ['None', 'l2'], # normal or ridge regression
    'C': [0.1, 1, 10]          # The inverse penalization term (smaller is higher penalization)
}

In [17]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_google_news, X_test_google_news, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

best parameters: {'C': 10, 'penalty': 'l2'}
Results for google_log_w:
Accuracy: 0.9151711378353377
Precision: 0.4944273511263321
Recall: 0.34781010828554143
F1: 0.40113627754245584


#### B. Decision Tree

In [20]:
# define the model characteristics
model_name = 'google_DT_w'
classifier = 'DT'

# Initialize the classifier
tree = DecisionTreeClassifier(random_state = 7)

# Define the parameter grid
param_grid_DT = {
    'criterion': ['gini'],          # Define the splitting criteria: Gini index for node impurity
    'min_samples_leaf': [1, 2],     # Define the minimum number of samples required to be at leaf node
    'max_features': [None]  # Define the number of features to consider when looking for the best split
}

In [21]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_google_news, X_test_google_news, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': None, 'min_samples_leaf': 1}
Results for google_DT_w:
Accuracy: 0.8480111008325625
Precision: 0.17567993031848514
Recall: 0.18983106229385277
F1: 0.18191737188918056


#### C. Support Vector Machine

In [22]:
# define the model characteristics
model_name = 'google_svm_w'
classifier = 'SVM'

# Initialize the classifier
svm = SVC(random_state = 7)

# Define the parameter grid
param_grid_svm = {
    'C': [0.1, 1, 10, 100], # inverse regularization parameter
    'kernel': ['linear', 'poly', 'rbf'], # what type of kernel need to be used (rbf = radial kernel)
}

In [23]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_google_news, X_test_google_news, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

best parameters: {'C': 10, 'kernel': 'rbf'}
Results for google_svm_w:
Accuracy: 0.9238667900092506
Precision: 0.5922298784096163
Recall: 0.4080671016213115
F1: 0.47197667821066425


#### D. Random Forest Classifier

In [24]:
# define the model characteristics
model_name = 'google_rf_w'
classifier = 'RF'

# Initialize the classifier
rfc = RandomForestClassifier(random_state = 7, n_jobs = -1)

# Define the parameter grid
param_grid_rf = {
    'criterion': ['gini'],          # Define the splitting criteria: Gini index for node impurity
    'n_estimators': [100, 500],     # the number of trees to use when building the model
    'min_samples_leaf': [1, 2],     # Define the minimum number of samples required to be at leaf node
    'max_features': ['sqrt']        # Define the number of features to consider when looking for the best split
}

In [25]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_google_news, X_test_google_news, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 100}
Results for google_rf_w:
Accuracy: 0.9049028677150787
Precision: 0.6042766768026284
Recall: 0.1183776120621839
F1: 0.15245559205058493


#### E. Adaboost classifier

In [28]:
# define the model characteristics
model_name = 'google_ada_w'
classifier = 'ADA'

# Initialize decision tree base estimator for AdaBoost
base_estimator = DecisionTreeClassifier(random_state = 7)

# Initialize AdaBoost classifier
ada = AdaBoostClassifier(base_estimator = base_estimator, random_state = 7)

# Define parameter grid for AdaBoost
param_grid_ada = {
    'n_estimators': [50, 100],   # the maximum number of estimators before boosting is terminated
    'learning_rate': [0.01, 0.1], # weight applied to each classifier at boosting iteration
                                      # A higher learning rate increases the contribution of each classifier. 
}

In [29]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_google_news, X_test_google_news, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

best parameters: {'learning_rate': 0.01, 'n_estimators': 100}
Results for google_ada_w:
Accuracy: 0.903422756706753
Precision: 0.40712695220892786
Recall: 0.14943771078499718
F1: 0.18998739893219468


### 3.1.2 With undersampling

In [30]:
# Define the resampling technique
resampling = 'Und'

In [31]:
# Define the categories and the maximum number of samples
categories = df_train["category"].unique()

#### Random undersampling

Again, we used the same undersampling strategy as we used in the notebook 'Vectorization'. We used random undersampling and reduced the imbalance ratio to 4:1 for each category.

#### Second strategy

In [32]:
# Calculate the number of samples in the biggest minority category
rus_n = df_train['category'].value_counts().sort_values(ascending=False)[1]

# Dictionary to store the actual maximum imbalance per class for undersampling
max_imbalance_u = {}

# Calculate the actual maximum imbalance for each class
for category in categories:
    if category == 'None':
        max_imbalance_u[category] = rus_n
    else:
        # Set the actual maximum to the number of available samples
        max_imbalance_u[category] = y_train.value_counts()[category]

#### Second strategy

In [33]:
# Create the random undersampler with maximum imbalance
undersampler = RandomUnderSampler(sampling_strategy = max_imbalance_u, random_state = 7)

# Undersample the data
X_train_google_news_und, y_train_google_news_und = undersampler.fit_resample(X_train_google_news, y_train)
#y_train_cbow_und.value_counts()

#### A. Logistic Regression

In [34]:
# define the model characteristics
model_name = 'google_log_u'
classifier = 'logR'

In [35]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_google_news_und, X_test_google_news,
                    y_train_google_news_und, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'C': 1, 'penalty': 'l2'}
Results for google_log_u:
Accuracy: 0.8086031452358927
Precision: 0.3003106892441716
Recall: 0.6290692503922747
F1: 0.3898184671430985


#### B. Decision Tree

In [36]:
# define the model characteristics
model_name = 'google_DT_u'
classifier = 'DT'

In [37]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_google_news_und, X_test_google_news,
                    y_train_google_news_und, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': None, 'min_samples_leaf': 1}
Results for google_DT_u:
Accuracy: 0.5360777058279371
Precision: 0.12199110229967723
Recall: 0.30212374019055194
F1: 0.13961655818685964


#### C. Support Vector Machine

In [38]:
# define the model characteristics
model_name = 'google_svm_u'
classifier = 'SVM'

In [39]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_google_news_und, X_test_google_news,
                    y_train_google_news_und, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'C': 10, 'kernel': 'rbf'}
Results for google_svm_u:
Accuracy: 0.8172062904717854
Precision: 0.3124788808488373
Recall: 0.6585206426374626
F1: 0.4059688202168255


#### D. Random Forest Classifier

In [40]:
# define the model characteristics
model_name = 'google_rf_u'
classifier = 'RF'

In [41]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_google_news_und, X_test_google_news,
                    y_train_google_news_und, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 500}
Results for google_rf_u:
Accuracy: 0.8000925069380204
Precision: 0.3754786739319368
Recall: 0.45722451044908935
F1: 0.3429789344635923


#### E. Adaboost classifier

In [42]:
# define the model characteristics
model_name = 'google_ada_u'
classifier = 'ADA'

In [43]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_google_news_und, X_test_google_news,
                    y_train_google_news_und, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'learning_rate': 0.1, 'n_estimators': 50}
Results for google_ada_u:
Accuracy: 0.6430157261794635
Precision: 0.15116101389775635
Recall: 0.3654870128378791
F1: 0.186797605690062


### 3.1.3 With oversampling

In [44]:
# Define the resampling technique
resampling = 'Ove'

#### Oversampling with SMOTE

Again, we used the same oversampling strategy as we used in the notebook 'Vectorization'. We used SMOTE and reduced the imbalance ratio to 4:1 for each category.

In [45]:
# Calculate the number of samples in the majority class
ove_n = df_train['category'].value_counts().sort_values(ascending=False)[0]

# Oversample until the number of observations equals a fourth of the majority class
max_samples = int(ove_n/4)

# Dictionary to store the actual maximum imbalance per class for oversampling
max_imbalance_o = {}

# Calculate the actual maximum imbalance for each class
for category in categories:
    if category == 'None':
        max_imbalance_o[category] = y_train.value_counts()[category]
    else:
        # Set the actual maximum to the number of available samples
        max_imbalance_o[category] = max_samples

In [46]:
# Create the SMOTE oversampler
oversampler = SMOTE(sampling_strategy=max_imbalance_o, random_state=7)

# Undersample the data
X_train_google_news_ove, y_train_google_news_ove = oversampler.fit_resample(X_train_google_news, y_train)
#y_train_skip_ove.value_counts()

#### A. Logistic Regression

In [47]:
# define the model characteristics
model_name = 'google_log_o'
classifier = 'logR'

In [48]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'C': 10, 'penalty': 'l2'}
Results for google_log_o:
Accuracy: 0.8451433857539316
Precision: 0.31730791885718557
Recall: 0.5710006988048183
F1: 0.3941828939075108


#### B. Decision Tree

In [49]:
# define the model characteristics
model_name = 'google_DT_o'
classifier = 'DT'

In [50]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': None, 'min_samples_leaf': 1}
Results for google_DT_o:
Accuracy: 0.7578168362627197
Precision: 0.156158291422305
Recall: 0.26608290100375365
F1: 0.1851608026753601


#### C. Support Vector Machine

In [51]:
# define the model characteristics
model_name = 'google_svm_o'
classifier = 'SVM'

In [52]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'C': 100, 'kernel': 'rbf'}
Results for google_svm_o:
Accuracy: 0.9226641998149862
Precision: 0.5666147580648718
Recall: 0.4113190138393927
F1: 0.4708754567420214


#### D. Random Forest Classifier

In [53]:
# define the model characteristics
model_name = 'google_rf_o'
classifier = 'RF'

In [54]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

best parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 500}
Results for google_rf_o:
Accuracy: 0.9181313598519889
Precision: 0.6295121905159177
Recall: 0.3108057864572282
F1: 0.38312343531159077


#### E. Adaboost classifier

In [55]:
# define the model characteristics
model_name = 'google_ada_o'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

## 3.2 Glove embeddings

In order to use the Glove embeddings, you first need to download their pretrained vectors on the website below. In our scenario, I used the vectors that were trained based on the Wikipedia 2014 and Gigaword 5. These vectors were trained on 6B tokens and a vocabulary of 400.000 words. It is possible to retrieve embeddins that were trained on a larger dataset. Moreover, these vectors come in 4 dimension. You can choose between 50, 100, 200 or 300 dimensions.

https://nlp.stanford.edu/projects/glove/

In [ ]:
# define vectorizer method
vectorizer = 'pretrained'

In [ ]:
# define type of pretrained embedding model
FS = 'Glove'

In [ ]:
# Convert the formats
glove_300_w2v = 'glove.6B.300d.text' + '.w2v'
glove2word2vec('/Users/juarel/Desktop/studies artur/thesis_HIR/big files/glove/glove.6B.300d.txt', glove_300_w2v)

In [ ]:
glove_300_w2v_model = KeyedVectors.load_word2vec_format(glove_300_w2v, binary=False)

In [ ]:
X_train_glove_300 = np.vstack([vectorize_pretrained(headline, glove_300_w2v_model, 300) for headline in X_train])
X_test_glove_300 = np.vstack([vectorize_pretrained(headline, glove_300_w2v_model, 300) for headline in X_test])

### 3.2.1 Without resampling

In [ ]:
# Define the resampling technique
resampling = 'None'

#### A. Logistic regression

In [ ]:
# define the model characteristics
model_name = 'glove_log_w'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_glove_300, X_test_glove_300, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'glove_DT_w'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_glove_300, X_test_glove_300, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'glove_svm_w'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_glove_300, X_test_glove_300, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'glove_rf_w'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_glove_300, X_test_glove_300, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'glove_ada_w'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_glove_300, X_test_glove_300, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

### 3.2.2 With undersampling

In [ ]:
# Define the resampling technique
resampling = 'Und'

#### Second strategy

In [ ]:
# Undersample the data
X_train_glove_300_und, y_train_glove_300_und = undersampler.fit_resample(X_train_glove_300, y_train)
#y_train_cbow_und.value_counts()

#### A. Logistic Regression

In [ ]:
# define the model characteristics
model_name = 'glove_log_u'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_glove_300_und, X_test_glove_300,
                    y_train_glove_300_und, y_test, vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'glove_DT_u'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_glove_300_und, X_test_glove_300,
                    y_train_glove_300_und, y_test, vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'glove_svm_u'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_glove_300_und, X_test_glove_300,
                    y_train_glove_300_und, y_test, vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'glove_rf_u'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_glove_300_und, X_test_glove_300,
                    y_train_glove_300_und, y_test, vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'glove_ada_u'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_glove_300_und, X_test_glove_300,
                    y_train_glove_300_und, y_test, vectorizer, FS, classifier, resampling)

### 3.2.3 With oversampling

In [ ]:
# Define the resampling technique
resampling = 'Ove'

In [ ]:
# Undersample the data
X_train_glove_300_ove, y_train_glove_300_ove = oversampler.fit_resample(X_train_glove_300, y_train)
#y_train_skip_ove.value_counts()

#### A. Logistic Regression

In [ ]:
# define the model characteristics
model_name = 'glove_log_o'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_glove_300_ove, X_test_glove_300,
                    y_train_glove_300_ove, y_test, vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'glove_DT_o'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_glove_300_ove, X_test_glove_300,
                    y_train_glove_300_ove, y_test, vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'glove_svm_o'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_glove_300_ove, X_test_glove_300,
                    y_train_glove_300_ove, y_test, vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'glove_rf_o'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_glove_300_ove, X_test_glove_300,
                    y_train_glove_300_ove, y_test, vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'glove_ada_o'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_glove_300_ove, X_test_glove_300,
                    y_train_glove_300_ove, y_test, vectorizer, FS, classifier, resampling)

## 3.3 Spacy model

https://spacy.io/models/en#en_core_web_lg

One of the shortcomings of the two previous pretrained models is that they are trained on data from 2013 (google news dataset) and 2014 (Glove embeddings). However, our data consists out headlines that were published between 2014 and 2020. Therefore, we also aimed to use a pretrained model based on more recent data. Therefore, we use the Spacy package. Spacy is a powerful natural language package that has four basic pipeline models to convert English words from text to embeddings:

1. en_core_web_sm
2. en_core_web_md
3. en_core_web_lg
4. en_core_web_trf

The sm (small), md (medium) and lg (large) models are all designed to run on a CPU (regular computer). The trf is transformer based (Roberta Model) and runs best on a GPU. 

In [ ]:
# In case it's not already downloaded, then run this code to download it # 
#spacy.cli.download("en_core_web_lg")

In [ ]:
# Load NLP Model from Spacy #
spacy_lg = spacy.load('en_core_web_lg')

In [ ]:
X_train_spacy_lg = np.vstack([spacy_lg(headline).vector.reshape(1,-1) for headline in X_train])
X_test_spacy_lg = np.vstack([spacy_lg(headline).vector.reshape(1,-1) for headline in X_test])

In [ ]:
# define vectorizer method
vectorizer = 'pretrained'

In [ ]:
# define type of pretrained embedding model
FS = 'Spacy'

### 3.3.1 Without resampling

In [ ]:
# Define the resampling technique
resampling = 'None'

#### A. Logistic regression

In [ ]:
# define the model characteristics
model_name = 'spacy_log_w'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_spacy_lg, X_test_spacy_lg, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'spacy_DT_w'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_spacy_lg, X_test_spacy_lg, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'spacy_svm_w'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_spacy_lg, X_test_spacy_lg, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_rf_w'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_spacy_lg, X_test_spacy_lg, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_ada_w'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_spacy_lg, X_test_spacy_lg, y_train, y_test,
                   vectorizer, FS, classifier, resampling)

### 3.3.2 With undersampling

In [ ]:
# Define the resampling technique
resampling = 'Und'

#### Second strategy

In [ ]:
# Undersample the data
X_train_spacy_und, y_train_spacy_und = undersampler.fit_resample(X_train_spacy_lg, y_train)
#y_train_cbow_und.value_counts()

#### A. Logistic Regression

In [ ]:
# define the model characteristics
model_name = 'spacy_log_u'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_spacy_und, X_test_spacy_lg,
                    y_train_spacy_und, y_test, vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'spacy_DT_u'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_spacy_und, X_test_spacy_lg,
                    y_train_spacy_und, y_test, vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'spacy_svm_u'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_spacy_und, X_test_spacy_lg,
                    y_train_spacy_und, y_test, vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_rf_u'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_spacy_und, X_test_spacy_lg,
                    y_train_spacy_und, y_test, vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_ada_u'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_spacy_und, X_test_spacy_lg,
                    y_train_spacy_und, y_test, vectorizer, FS, classifier, resampling)

### 3.3.3 With oversampling

In [ ]:
# Define the resampling technique
resampling = 'Ove'

In [ ]:
# Undersample the data
X_train_spacy_ove, y_train_spacy_ove = oversampler.fit_resample(X_train_spacy_lg, y_train)
#y_train_skip_ove.value_counts()

#### A. Logistic Regression

In [ ]:
# define the model characteristics
model_name = 'spacy_log_o'
classifier = 'logR'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, logreg, param_grid_log, X_train_spacy_ove, X_test_spacy_lg,
                    y_train_spacy_ove, y_test, vectorizer, FS, classifier, resampling)

#### B. Decision Tree

In [ ]:
# define the model characteristics
model_name = 'spacy_DT_o'
classifier = 'DT'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, tree, param_grid_DT, X_train_spacy_ove, X_test_spacy_lg,
                    y_train_spacy_ove, y_test, vectorizer, FS, classifier, resampling)

#### C. Support Vector Machine

In [ ]:
# define the model characteristics
model_name = 'google_svm_o'
classifier = 'SVM'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, svm, param_grid_svm, X_train_google_news_ove, X_test_google_news,
                    y_train_google_news_ove, y_test, vectorizer, FS, classifier, resampling)

#### D. Random Forest Classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_rf_o'
classifier = 'RF'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, rfc, param_grid_rf, X_train_spacy_ove, X_test_spacy_lg,
                    y_train_spacy_ove, y_test, vectorizer, FS, classifier, resampling)

#### E. Adaboost classifier

In [ ]:
# define the model characteristics
model_name = 'spacy_ada_o'
classifier = 'ADA'

In [ ]:
# perform a grid search for the logistic regression model
# the results are automatically stored in results_all_df and best_params_dict
perform_grid_search(model_name, ada, param_grid_ada, X_train_spacy_ove, X_test_spacy_lg,
                    y_train_spacy_ove, y_test, vectorizer, FS, classifier, resampling)

## 4. Write away results

In [ ]:
# write away results
results_all_df.to_csv('./Output/Model performance/results_pretrained_embeddings.csv', index = False, header = True)

In [ ]:
# Write the dictionary with the best parameters away
with open('./Output/parameters/pretrained_embeddings.json', 'w') as file:
    json.dump(best_params_dict, file)